# Reader Study

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import tqdm

/home/bdrad/miniconda3/envs/report_gen/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def clean_date(timestamp):
    dt = pd.Timestamp(timestamp)
    return f"{dt.strftime('%B')} {dt.day}, {dt.year} - {dt.strftime('%I:%M %p')}"

def generate_note_title(enc_dept_name, note_type, auth_prov_type, deid_service_date):
    deid_service_date = clean_date(deid_service_date) if deid_service_date else ""
    if enc_dept_name and note_type and auth_prov_type and deid_service_date:
        return f"{enc_dept_name} - {note_type} ({auth_prov_type}) | {deid_service_date}"
    elif enc_dept_name and note_type and deid_service_date:
        return f"{enc_dept_name} - {note_type} | {deid_service_date}"
    elif note_type and deid_service_date:
        return f"{note_type} | {deid_service_date}"
    elif deid_service_date:
        return f"Clinical Note | {deid_service_date}"
    return f"Clinical Note"

def generate_note_titles(row):
    return [generate_note_title(ed, nt, apt, ds)
            for ed, nt, apt, ds in zip(row['enc_dept_names'],
                                        row['note_types'],
                                        row['auth_prov_types'],
                                        row['deid_service_dates'])]

# Reader Study Evaluation Dataset

In [8]:
reader_evaluation_dataset = pd.read_parquet("reader_evaluation_dataset.parquet")
reader_evaluation_dataset["note_titles"] = reader_evaluation_dataset.apply(generate_note_titles, axis=1)

claude_evaluation = pd.read_csv("../old_inference/results/reader_evaluation_dataset/claude3_5_0_250.csv")[["llm_indication"]].rename(
    columns={"llm_indication": "llm_indication_claude"}
)

llama_evaluation = pd.read_csv("../old_inference/results/reader_evaluation_dataset/Qwen_Qwen2.5-7B-Instruct_0_250.csv")[["llm_indication"]].rename(
    columns={"llm_indication": "llm_indication_qwen"}
)

reader_evaluation_dataset = pd.concat([reader_evaluation_dataset, claude_evaluation, llama_evaluation], axis=1)

In [6]:
SEED = 123
indication_choices = [
    "original_history", 
    "additional_history", 
    "llm_indication_claude",
    "llm_indication_qwen"
]
def randomize_indications(_):
    choices = indication_choices.copy()
    np.random.shuffle(choices)
    return pd.Series(choices, index=["indication1_random", "indication2_random", "indication3_random", "indication4_random"])

# Apply the function row-wise and assign the new columns to the DataFrame
reader_evaluation_dataset[["indication1_choice", "indication2_choice", "indication3_choice", "indication4_choice"]] = \
    reader_evaluation_dataset.apply(randomize_indications, axis=1)

reader_evaluation_dataset.to_parquet("randomized_reader_study_evaluation.parquet")

In [11]:
reader_evaluation_dataset.head()

,patientdurablekey,radiology_deid_note_key,exam_type,radiology_text,radiology_deid_service_date,original_history,additional_history,enc_dept_names,note_types,auth_prov_types,deid_service_dates,note_texts,pathophysiological_category,imaging_modality,body_system,radiology_report_date,note_titles,llm_indication_claude,llm_indication_qwen
0,D4364B98A40C01,D588D31D57F14F,CT CHEST WITHOUT CONTRAST,CT CHEST WITHOUT CONTRAST 03/07/2024 5:47 P...,2024-03-08 04:10:00,"s/p lung tx, one month study.",Status post bilateral lung transplant on 02/06...,"[10LS CVT, 10LS CVT, 10LS CVT, 10LS CVT, 10LS ...","[Consult, Consults, Consults, Consults, Consul...","[Nurse Practitioner, Resident, Registered Diet...","[2024-03-07T14:19:00.000000, 2024-03-07T13:40:...","[***** *****, MD 03/07/2024 9:30 PM""\n\ne...",Surgical,CT,Chest,2024-03-07 17:47:00,[10LS CVT - Consult (Nurse Practitioner) | Mar...,BLTx on 2/6/24 complicated by left pectoral he...,"Post-lung transplant, 1 month follow-up."
1,D0E045633AE903,D4A821CB534C52,CT CHEST PULMONARY EMBOLISM (CTPE),CT CHEST PULMONARY EMBOLISM (CTPE) 02/15/20...,2023-02-15 04:05:00,Pulmonary embolism suspected,"s/p revision TKA on 02/14/2023, developed hypo...","[MZ 4 WEST ICU, MZ 4 WEST ICU, MZ 4 WEST ICU, ...","[RN Note, Significant Event, H&P, Consults, Co...","[Registered Nurse, Resident, Physician, Physic...","[2023-02-15T02:35:00.000000, 2023-02-15T02:15:...",[0100 - Patient's HR showed 45 on CPO. This RN...,Surgical,CT,Chest,2023-02-15 02:50:00,[MZ 4 WEST ICU - RN Note (Registered Nurse) | ...,Evaluate for suspected pulmonary embolism post...,Suspected pulmonary embolism following right T...
2,D33811103BFCE7,D9A8DAAB19FE54,MR ANGIOGRAM NECK WITH AND WITHOUT CONTRAST,MR ANGIOGRAM NECK WITH AND WITHOUT CONTRAST: ...,2023-07-26 00:38:00,abnormal spinal varices,Evaluate for any possible mass lesion or vascu...,"[C5 P TRANS MB, C5 P TRANS MB, C5 P TRANS MB, ...","[Consults, Interdisciplinary, ED Provider Note...","[Resident, Social Worker, Resident, None, None...","[2023-07-25T21:43:00.000000, 2023-07-25T14:13:...",[PEDIATRIC NEUROSURGERY INITIAL INPATIENT CONS...,Cancer/Mass,MRI,Neck,2023-07-25 22:23:00,[C5 P TRANS MB - Consults (Resident) | July 25...,10-year-old male with thoracic back pain and L...,Pediatric patient with congenital heart diseas...
3,D93E9B80604E0E,D27F4753CBBB35,CT SINUSES WITHOUT CONTRAST,CT SINUSES WITHOUT CONTRAST: 05/18/2023 10:13...,2023-05-19 00:15:00,"preop planning, OR on 05/19/23","Multiple presumed intracranial metastases, inc...","[8L NEUROSCIENCES, NEUROSURG ADLT PARN, NEUROS...","[H&P, Progress Notes, Letter, Anesthesia Prepr...","[Resident, Physician Assistant, None, Resident...","[2023-05-18T19:06:00.000000, 2023-05-15T00:00:...",[NEUROSURGERY HISTORY & PHYSICAL NOTE HPI ...,Surgical,CT,Neck,2023-05-18 22:13:00,"[8L NEUROSCIENCES - H&P (Resident) | May 18, 2...","Suprasellar mass, newly diagnosed lung adenoca...","Female, 82 y.o., with systemic sclerosis, lung..."
4,DEE13C33F1DDB9,DD1CFEE4F5D246,MR ORBIT WITH AND WITHOUT CONTRAST,MR ORBIT WITH AND WITHOUT CONTRAST: 01/20/202...,2021-01-21 10:05:00,"Other reason, no headache, head trauma, seizur...",7 year history of remitting bilateral idiopath...,"[None, OPHTH PLASTIC MB 490 FL3, None, OPHTH G...","[HNO_MYC_LET_INFO|NOTE_ID, Progress Notes, HNO...","[None, Physician, None, Physician, Physician A...","[2021-01-15T23:30:00.000000, 2021-01-15T00:00:...",[COVID-19 Screening Result 01/15/21 CLEA...,Infection/Inflammatory,MRI,Neck,2021-01-20 13:49:00,"[HNO_MYC_LET_INFO|NOTE_ID | January 15, 2021 -...",53-year-old female with 7-year history of rela...,Bilateral intermittent orbital inflammation wi...


In [12]:
reader_evaluation_dataset[["patientdurablekey"]].to_csv("CTSI_deidentified_mrn.csv", index=False)

# Case Assignment

In [20]:
#create small groupings, this is necessary for uniform distribution to ensure equal group sizing (in setting of a rectangular matrix...)
# we want 25 cases per person for 20 people  , 2 repeats of each case , so we want a dimension 10 matrix , 5 groupings of 3 and 5 groupings of 2
X = list()
for i in range(10):
    row = list()
    for j in range(10):
        if (j - i) % 10 in [0, 1, 2, 3, 4]:
            row.append(3)
        else:
            row.append(2)
    X.append(row)

# use X and our index array to create a matrix of the actual cases, grouped according to small groups defined in X - call this matrix Y.
# essentially we are sampling from the case index (labeled index) and drawing # of cases according to matrix X
# well its a "matrix" but really the structure is a list of lists. the first index will refer to the "row" and the second index will be the "column"
# Y will be a square matrix by construction

index = list(range(250))
Y = list()
start_index=0

for i in range(10):
  row = list()
  for j in range(10):
    row.append(index[start_index:start_index+X[i][j]])
    #update start_index
    start_index=start_index+X[i][j]
  Y.append(row)

# use matrix Y to create the final assignments by taking the union across each row to create the first set of 10 cases and then do the same by column for the second set of 10 cases store these as arrays

case_assignments=list()

#first assignment for first 10 raters, read along rows of Y
for i in range(10):
  case_assignments.append([element for row in Y[i] for element in row])

#second assignment for raters 11-20, read along columns of Y, create array of arrays to store reworked values. unfortunately list of lists makes this hard so create a temporary matrix Y2 first which is a transpose of Y
Y2= [Y[j][i] for i in range(10) for j in range(10)]

for i in range(10):
  case_assignments.append(([element for row in Y2[10*i:10*i+10] for element in row]))

In [21]:
import pandas as pd

NUM_READERS = 20
WORDS_PER_LINE = 10 
LINES_PER_BLOCK = 20


for i in tqdm.tqdm(range(NUM_READERS)):
    case_assignment = case_assignments[i]
    reader_evaluation_dataset_subset = reader_evaluation_dataset.iloc[case_assignment].reset_index(drop=True)
    for j in range(len(reader_evaluation_dataset_subset)):
        row = reader_evaluation_dataset_subset.iloc[j]
        basepath = f"/mnt/sohn2022/Adrian/rad-llm-pmhx/dataset/reader_study/evaluation/public/user{i+1}/set{j+1}"
        os.makedirs(basepath, exist_ok=True)
        
        with open(f"{basepath}/exam.txt", "w", encoding="utf-8") as f:
            f.write(str(row["exam_type"]))
        
        note_titles = row["note_titles"]
        note_texts = row["note_texts"]
        
        for k in range(10):
            filename = f"{basepath}/note{k+1}.txt"
            with open(filename, "w", encoding="utf-8") as f:
                if k < len(note_titles) and k < len(note_texts):
                    f.write(note_titles[k] + "\n")
                    words = note_texts[k].split()
                    line_count = 0
                    for idx in range(0, len(words), WORDS_PER_LINE):
                        line = " ".join(words[idx:idx + WORDS_PER_LINE])
                        f.write(line + "\n")
                        line_count += 1
                        if line_count % LINES_PER_BLOCK == 0:
                            f.write("\n") 
                else:
                    f.write("")
        
        for l in range(4):
            filename = f"{basepath}/indication{l+1}.txt"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(row[row[f"indication{l+1}_choice"]])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.49s/it]
